In [1]:
from lmfit import Parameters
from lmfit import minimize, fit_report

In [2]:
import pyroll.core as pr
import pyroll.pillar_model
import pyroll.neutral_point_estimator
import pyroll.sparling_spreading
import pyroll.hensel_force_torque
import pyroll.freiberg_flow_stress
import pyroll.ring_model_thermal

In [3]:
from in_profile import create_in_profile_from_surface_temperature
from rolling_sequence import rolling_train_rpd1, rolling_train_rpd2, rolling_train_rpd3
from utilities import *

In [4]:
pyroll.pillar_model.Config.PILLAR_COUNT = 30
pyroll.neutral_point_estimator.Config.ESTIMATOR = "EQUAL"
ROLL_SURFACE_TEMPERATURE = 50 + 273.15
DISK_ELEMENT_COUNT = 30

In [5]:
@pr.CoolingPipe.DiskElement.heat_transfer_coefficient
def heat_transfer_coefficient_model_moreales(self: pr.CoolingPipe.DiskElement):
    mean_surface_temperature = (self.in_profile.surface_temperature + self.out_profile.surface_temperature) / 2
    profile_surface_temperature_celsius = mean_surface_temperature - 273.15
    return 13950 + 5.12 * 1e6 / profile_surface_temperature_celsius

In [6]:
in_profile = create_in_profile_from_surface_temperature(1050 + 273.15)
in_profile.ring_temperatures = in_profile.ring_temperatures

In [7]:
rolling_train_rpd1 = rolling_train_rpd1(ROLL_SURFACE_TEMPERATURE, DISK_ELEMENT_COUNT)
rolling_train_rpd2 = rolling_train_rpd2(ROLL_SURFACE_TEMPERATURE, DISK_ELEMENT_COUNT)
rolling_train_rpd3 = rolling_train_rpd3(ROLL_SURFACE_TEMPERATURE, DISK_ELEMENT_COUNT)

In [8]:
rolling_train_rpd1.solve(in_profile)
rolling_train_rpd2.solve(in_profile)
rolling_train_rpd3.solve(in_profile)

Solution iteration of TwoRollPass '3' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of TwoRollPass '7' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of TwoRollPass '8' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of TwoRollPass '14' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of TwoRollPass '18-V' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of TwoRollPass '23-H' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of TwoRollPass '18-V' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of TwoRollPass '3' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of TwoRollPass '7' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of TwoRollPass '8' exceeded the maximum iteration cou

Profile(
    classifiers={'false_round', 'generic_elongation', 'round', 'symmetric'},
    core_temperature=np.float64(1563.6262441102974),
    cross_section=Polygon(
        area=5.893659124183413e-05,
        height=0.008700000000000006,
        perimeter=0.027240114871976685,
        width=0.008517743190215882,
        ),
    length=5211.114927918349,
    material=['BST500', 'steel'],
    ring_temperatures=array([1563.62624411, 1560.75020294, 1552.12609703, 1537.75030537,
           1517.57709745, 1491.58658858, 1460.14304003, 1424.77487668,
           1389.10595613, 1358.98549405, 1340.35438669]),
    strain=np.float64(0.5177718111612112),
    surface_temperature=np.float64(1340.3543866900577),
    t=np.float64(36.838988272772404),
    temperature=np.float64(1423.7380968234704),
    weight=2400,
    width=np.float64(0.008517743190215882),
    )

In [9]:
sequences = [
        {
            "label": "RPD1",
            "sequence": rolling_train_rpd1
        },
        {
            "label": "RPD2",
            "sequence": rolling_train_rpd2
        },
        {
            "label": "RPD3",
            "sequence": rolling_train_rpd3
        }
    ]

In [26]:
def solve_total_wear_depths_as_contour_line(params):
    area_errors = []

    for entry in sequences:
        groove_label = entry["label"]
        sequence = entry["sequence"]
        roll_passes = sequence.roll_passes
        measurements = extract_wear_contours_from_measurement(groove_label)

        for measurement_id, labels, tonnages, measured_wear_contours in measurements:
            for roll_pass in roll_passes:
                for label, tonnage, measured_wear_contour in zip(labels, tonnages, measured_wear_contours):
                    if roll_pass.label == label:

                        wear_coefficient = get_wear_coefficient(roll_pass, params)
                        calculated_wear_contour = calculate_wear_contour(roll_pass, wear_coefficient, tonnage * 1000)
                        area_error = calculate_area_between_contours(calculated_wear_contour, measured_wear_contour)
                        area_errors.append(area_error)
                    else:
                        continue
    rms_error = root_mean_square_value(np.array(area_errors))
    print("RMS error: ", rms_error)
    return rms_error


In [11]:
def solve_total_wear_depths_as_contour_line_4_pass_block(params):
    area_errors = []

    for entry in sequences:
        groove_label = entry["label"]
        sequence = entry["sequence"]
        roll_passes = sequence.roll_passes
        measurements = extract_wear_contours_from_measurement(groove_label)

        for measurement_id, labels, tonnages, measured_wear_contours in measurements:
            for roll_pass in roll_passes:
                for label, tonnage, measured_wear_contour in zip(labels, tonnages, measured_wear_contours):
                    if roll_pass.label == label and label in ["17-H", "18-V"]:

                        wear_coefficient = get_wear_coefficient(roll_pass, params)
                        calculated_wear_contour = calculate_wear_contour(roll_pass, wear_coefficient, tonnage * 1000)
                        area_error = calculate_area_between_contours(calculated_wear_contour, measured_wear_contour)
                        area_errors.append(area_error)
                    else:
                        continue
    rms_error = root_mean_square_value(np.array(area_errors))
    return rms_error


def solve_total_wear_depths_as_contour_line_6_pass_block(params):
    area_errors = []

    for entry in sequences:
        groove_label = entry["label"]
        sequence = entry["sequence"]
        roll_passes = sequence.roll_passes
        measurements = extract_wear_contours_from_measurement(groove_label)

        for measurement_id, labels, tonnages, measured_wear_contours in measurements:
            for roll_pass in roll_passes:
                for label, tonnage, measured_wear_contour in zip(labels, tonnages, measured_wear_contours):
                    if roll_pass.label == label and label in ["21-H", "22-V", "23-H", "24-V"]:

                        wear_coefficient = get_wear_coefficient(roll_pass, params)
                        calculated_wear_contour = calculate_wear_contour(roll_pass, wear_coefficient, tonnage * 1000)
                        area_error = calculate_area_between_contours(calculated_wear_contour, measured_wear_contour)
                        area_errors.append(area_error)
                    else:
                        continue
    rms_error = root_mean_square_value(np.array(area_errors))
    return rms_error


def solve_total_wear_depths_as_contour_line_csb_block(params):
    area_errors = []

    for entry in sequences:
        groove_label = entry["label"]
        sequence = entry["sequence"]
        roll_passes = sequence.roll_passes
        measurements = extract_wear_contours_from_measurement(groove_label)

        for measurement_id, labels, tonnages, measured_wear_contours in measurements:
            for roll_pass in roll_passes:
                for label, tonnage, measured_wear_contour in zip(labels, tonnages, measured_wear_contours):
                    if roll_pass.label == label and label in ["27-H", "28-V"]:

                        wear_coefficient = get_wear_coefficient(roll_pass, params)
                        calculated_wear_contour = calculate_wear_contour(roll_pass, wear_coefficient, tonnage * 1000)
                        area_error = calculate_area_between_contours(calculated_wear_contour, measured_wear_contour)
                        area_errors.append(area_error)
                    else:
                        continue
    rms_error = root_mean_square_value(np.array(area_errors))
    return rms_error

In [12]:
params_constant_cobyla = Parameters()
params_constant_cobyla.add('wear_coefficient', value=1e-5, min=1e-10, max=1e-1, vary=True)

params_linear_function_cobyla = Parameters()
params_linear_function_cobyla.add('base_wear_coefficient', value=1e-5, min=1e-10, max=1e-1, vary=True)
params_linear_function_cobyla.add('roll_gap_ratio_coefficient', value=1e-5, min=1e-10, max=1e-1, vary=True)

params_cubic_function_cobyla = Parameters()
params_cubic_function_cobyla.add('base_wear_coefficient', value=1e-5, min=1e-10, max=1e-1, vary=True)
params_cubic_function_cobyla.add('roll_gap_ratio_coefficient', value=1e-5, min=1e-10, max=1e-1, vary=True)
params_cubic_function_cobyla.add('speed_ratio_coefficient', value=1e-5, min=1e-10, max=1e-1, vary=True)

In [27]:
result_constant_cobyla = minimize(
    solve_total_wear_depths_as_contour_line,
    params_constant_cobyla,
    nan_policy='omit',
    max_nfev=100,
    method="cobyla"
)

RMS error:  0.019222878005880895
RMS error:  447.04068788575125
RMS error:  436.8091431866677
RMS error:  114.7839574684949
RMS error:  31.407703795350205
RMS error:  6.761609126357302
RMS error:  2.2758687122576
RMS error:  0.29861234452847735
RMS error:  0.23155546890968387
RMS error:  0.0010631472247207327
RMS error:  0.041576987359764166
RMS error:  0.013985903128088064
RMS error:  0.0001032324480466241
RMS error:  0.0028087795233483963
RMS error:  0.0009972296580702368
RMS error:  1.3108579834581784e-05
RMS error:  0.0001250180203165213
RMS error:  3.559059966790259e-05
RMS error:  1.3146438115049403e-05
RMS error:  1.4400570632908026e-05
RMS error:  1.4400570632908026e-05


In [23]:
result_constant_cobyla

In [13]:

result_linear_function_cobyla = minimize(
    solve_total_wear_depths_as_contour_line,
    params_linear_function_cobyla,
    nan_policy='omit',
    max_nfev=100,
    method="cobyla"
)

result_cubic_function_cobyla = minimize(
    solve_total_wear_depths_as_contour_line,
    params_cubic_function_cobyla,
    nan_policy='omit',
    max_nfev=100,
    method="cobyla"
)

In [14]:
print(fit_report(result_constant_cobyla))

[[Fit Statistics]]
    # fitting method   = COBYLA
    # function evals   = 20
    # data points      = 1
    # variables        = 1
    chi-square         = 2.0738e-10
    reduced chi-square = 2.0738e-10
    Akaike info crit   = -20.2964854
    Bayesian info crit = -22.2964854
##  Warning: uncertainties could not be estimated:
    this fitting method does not natively calculate uncertainties
    and numdifftools is not installed for lmfit to do this. Use
    `pip install numdifftools` for lmfit to estimate uncertainties
    with this fitting method.
[[Variables]]
    wear_coefficient:  4.8073e-10 (init = 1e-06)


In [15]:
print(fit_report(result_linear_function_cobyla))

[[Fit Statistics]]
    # fitting method   = COBYLA
    # function evals   = 29
    # data points      = 1
    # variables        = 2
    chi-square         = 2.4379e-10
    reduced chi-square = 2.4379e-10
    Akaike info crit   = -18.1347125
    Bayesian info crit = -22.1347125
##  Warning: uncertainties could not be estimated:
    this fitting method does not natively calculate uncertainties
    and numdifftools is not installed for lmfit to do this. Use
    `pip install numdifftools` for lmfit to estimate uncertainties
    with this fitting method.
[[Variables]]
    base_wear_coefficient:       1.0034e-10 (init = 1e-06)
    roll_gap_ratio_coefficient:  2.5243e-10 (init = 1e-06)


In [16]:
print(fit_report(result_cubic_function_cobyla))

[[Fit Statistics]]
    # fitting method   = COBYLA
    # function evals   = 44
    # data points      = 1
    # variables        = 3
    chi-square         = 5.7343e-10
    reduced chi-square = 5.7343e-10
    Akaike info crit   = -15.2793780
    Bayesian info crit = -21.2793780
##  Warning: uncertainties could not be estimated:
    this fitting method does not natively calculate uncertainties
    and numdifftools is not installed for lmfit to do this. Use
    `pip install numdifftools` for lmfit to estimate uncertainties
    with this fitting method.
[[Variables]]
    base_wear_coefficient:       2.8960e-10 (init = 1e-06)
    roll_gap_ratio_coefficient:  4.0208e-10 (init = 1e-06)
    speed_ratio_coefficient:     9.5768e-06 (init = 1e-06)


In [17]:
params_4_pass_block_cobyla = Parameters()
params_4_pass_block_cobyla.add('wear_coefficient', value=1e-6, min=1e-10, max=1e-1, vary=True)

params_6_pass_block_cobyla = Parameters()
params_6_pass_block_cobyla.add('wear_coefficient', value=1e-6, min=1e-10, max=1e-1, vary=True)

params_csb_block_cobyla = Parameters()
params_csb_block_cobyla.add('wear_coefficient', value=1e-6, min=1e-10, max=1e-1, vary=True)


result_4_pass_block_cobyla = minimize(
    solve_total_wear_depths_as_contour_line_4_pass_block,
    params_4_pass_block_cobyla,
    nan_policy='omit',
    max_nfev=100,
    method="cobyla"
)

result_6_pass_block_cobyla = minimize(
    solve_total_wear_depths_as_contour_line_6_pass_block,
    params_6_pass_block_cobyla,
    nan_policy='omit',
    max_nfev=100,
    method="cobyla"
)

result_csb_block_cobyla = minimize(
    solve_total_wear_depths_as_contour_line_csb_block,
    params_csb_block_cobyla,
    nan_policy='omit',
    max_nfev=100,
    method="cobyla"
)

In [18]:
print(fit_report(result_4_pass_block_cobyla))

[[Fit Statistics]]
    # fitting method   = COBYLA
    # function evals   = 20
    # data points      = 1
    # variables        = 1
    chi-square         = 4.3085e-10
    reduced chi-square = 4.3085e-10
    Akaike info crit   = -19.5652705
    Bayesian info crit = -21.5652705
##  Warning: uncertainties could not be estimated:
    this fitting method does not natively calculate uncertainties
    and numdifftools is not installed for lmfit to do this. Use
    `pip install numdifftools` for lmfit to estimate uncertainties
    with this fitting method.
[[Variables]]
    wear_coefficient:  4.8073e-10 (init = 1e-06)


In [19]:
print(fit_report(result_6_pass_block_cobyla))

[[Fit Statistics]]
    # fitting method   = COBYLA
    # function evals   = 20
    # data points      = 1
    # variables        = 1
    chi-square         = 1.8180e-10
    reduced chi-square = 1.8180e-10
    Akaike info crit   = -20.4280943
    Bayesian info crit = -22.4280943
##  Warning: uncertainties could not be estimated:
    this fitting method does not natively calculate uncertainties
    and numdifftools is not installed for lmfit to do this. Use
    `pip install numdifftools` for lmfit to estimate uncertainties
    with this fitting method.
[[Variables]]
    wear_coefficient:  4.8073e-10 (init = 1e-06)


In [21]:
print(fit_report(result_csb_block_cobyla))

[[Fit Statistics]]
    # fitting method   = COBYLA
    # function evals   = 20
    # data points      = 1
    # variables        = 1
    chi-square         = 3.5053e-11
    reduced chi-square = 3.5053e-11
    Akaike info crit   = -22.0741699
    Bayesian info crit = -24.0741699
##  Warning: uncertainties could not be estimated:
    this fitting method does not natively calculate uncertainties
    and numdifftools is not installed for lmfit to do this. Use
    `pip install numdifftools` for lmfit to estimate uncertainties
    with this fitting method.
[[Variables]]
    wear_coefficient:  4.8073e-10 (init = 1e-06)
